Source code of every test for the task A with a BERT model

In [1]:
import numpy as np
import pandas as pd

# Load CSV files.
# CSV task B
def getData():
    df_train_data = pd.read_csv("data/Training_Data/subtaskB_data_all.csv")
    df_train_answers = pd.read_csv("data/Training_Data/subtaskB_answers_all.csv")

    df_train = pd.merge(df_train_data,df_train_answers,on='id', how='left').drop(['id'], axis=1)
    
    df_dev_data = pd.read_csv("data/Dev_Data/subtaskB_dev_data.csv")
    df_dev_answers = pd.read_csv("data/Dev_Data/subtaskB_gold_answers.csv")

    df_dev = pd.merge(df_dev_data,df_dev_answers,on='id', how='left').drop(['id'], axis=1)

    df_test_data = pd.read_csv("data/Test_Data/subtaskB_test_data.csv")
    df_test_answers = pd.read_csv("data/Test_Data/subtaskB_gold_answers.csv")

    df_test= pd.merge(df_test_data,df_test_answers,on='id', how='left').drop(['id'], axis=1)
    
    return df_train, df_dev, df_test

df_train_B, df_dev_B, df_test_B = getData()

df_train_B.head()

,FalseSent,OptionA,OptionB,OptionC,answer
0,He poured orange juice on his cereal.,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...,B
1,He drinks apple.,Apple juice are very tasty and milk too,Apple can not be drunk,Apple cannot eat a human,B
2,"Jeff ran 100,000 miles today","100,000 miles is way to long for one person to...","Jeff is a four letter name and 100,000 has six...","100,000 miles is longer than 100,000 km.",A
3,I sting a mosquito,A human is a mammal,A human is omnivorous,A human has not stings,C
4,A giraffe is a person.,Giraffes can drink water from a lake.,A giraffe is not a human being.,.Giraffes usually eat leaves.,B


In [18]:
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk
from nltk.stem.porter import *

stemmer = PorterStemmer()

Methods to pre-process the dataframe

In [24]:
def lemmatizer(text):
    """
    Receives a string as an input and lemmatizes it.
    """
    str = ""
    doc = nlp(text)
    for token in doc:
        str+=" "+token.lemma_
    return str 


def stemmatizer(text) :
    """
    Receive a string in input and stem it.
    """
    str = ""
    doc = nlp(text)
    for token in doc :
        str += ""+stemmer.stem(token.text)
    return str

def removeStopWords(text):
    """
    Receives a string and remove stop words from it.
    """
    str = ""
    doc = nlp(text)
    for token in doc:
        if(not token.is_stop):
            str+=" "+token.text
    return str 

def ngrams(text, n):
    """
    Receives a text of tokens and generates n-grams.
    """
    sequence=[]
    str = ""
    doc = nlp(text)
    for token in doc :
        sequence.append(token.text)
    return list(tuple([sequence[i] for i in range(i, i+n)]) for i in range(len(sequence)-n+1))


In [4]:
def pre_process(df, function):
    newdf = df[['FalseSent', 'OptionA', 'OptionB', 'OptionC']]
    newdf.loc[:,"FalseSent"] = df.FalseSent.apply(function)
    newdf.loc[:,"OptionA"] = df.OptionA.apply(function)
    newdf.loc[:,"OptionB"] = df.OptionB.apply(function)
    newdf.loc[:,"OptionC"] = df.OptionC.apply(function)
    return newdf

Process of data frame, create subsample of it

In [5]:
def subsampleData():
    # subsample data 
    train = df_train_B.sample(n=1000, random_state=42)

    X_train = train[['FalseSent', 'OptionA', 'OptionB', 'OptionC']]
    y_train = train['answer']

    # use the dev set for testing  
    return X_train, y_train

X_test = df_dev_B[['FalseSent', 'OptionA', 'OptionB', 'OptionC']]
y_test = df_dev_B['answer']

In [6]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
def test_performance(model, x_test, y_test):
    y_pred = model.predict(x_test)
    print(classification_report(y_pred=y_pred, y_true=y_test))
    return f1_score(y_pred=y_pred, y_true=y_test, average="macro"), f1_score(y_pred=y_pred, y_true=y_test, average="micro")

Importation of the BERT model

In [7]:
from transformers import BertModel
from bert_sklearn import BertClassifier

In [8]:
model = BertClassifier(max_seq_length=64, train_batch_size=16)
#model.num_mlp_layers = 3
model.max_seq_length = 64
model.epochs = 3
#model.learning_rate = 4e-5
                             
model

Building sklearn text classifier...


BertClassifier(max_seq_length=64, train_batch_size=16)

In [9]:
X_train_sample, y_train= subsampleData()

Fit with different preprocess type                                                                

In [10]:
X_train_sample.head()

,FalseSent,OptionA,OptionB,OptionC
6252,a duck walks on three legs,the duck's legs are short,a duck has only two legs,a duck has two wings
4684,Jack's mom praised him because he broke the plate,"Breaking a plate is not a good thing, people w...","Plates are easy to break, people need to be ca...",Plates can be made of ceramic or plastic
1731,People use electricity to buy things,It is impossible to buy things with electricity,Electricity is essential to live,Many appliances in home works on electricity
4742,"The display is damaged, thus I can't hear anyt...",Display can also be used to create sound with ...,Display cannot be used to aid people's hearing,"Display is used to display things, people will..."
4521,Santa Claus is the legend of the East,Christmas is very grand in the West,The origin of Christmas is not in the East,Western countries are very respectful of Santa...


In [11]:
model_classic = model.fit(X_train_sample, y_train)
f1macro, f1micro = test_performance(model_classic, X_test, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Training  :   0%|                                                                               | 0/57 [00:00<?, ?it/s]C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\bert_sklearn\model\pytorch_pretrained\optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:1485.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:36<00:00,  2.84s/it]

Epoch 1, Train loss: 1.0667, Val loss: 0.9957, Val accy: 53.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.73s/it]

Epoch 2, Train loss: 0.8551, Val loss: 0.9771, Val accy: 55.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.71s/it]

Epoch 3, Train loss: 0.5921, Val loss: 1.0101, Val accy: 56.00%



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:37<00:00,  1.26s/it]

              precision    recall  f1-score   support

           A       0.67      0.65      0.66       344
           B       0.38      0.28      0.32       327
           C       0.43      0.56      0.48       326

    accuracy                           0.50       997
   macro avg       0.49      0.49      0.49       997
weighted avg       0.49      0.50      0.49       997

f1micro = 0.496 and f1macro = 0.488


With only lemma

In [12]:
X_train = pre_process(X_train_sample,lemmatizer)
X_train.head()

X_test_lemma = pre_process(X_test,lemmatizer)

In [13]:
model_lemma = model.fit(X_train, y_train)
f1macro, f1micro = test_performance(model_lemma, X_test, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")
f1macro, f1micro = test_performance(model_lemma, X_test_lemma, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:36<00:00,  2.81s/it]

Epoch 1, Train loss: 1.0588, Val loss: 0.9912, Val accy: 54.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.73s/it]

Epoch 2, Train loss: 0.8407, Val loss: 0.9203, Val accy: 64.00%


...

Epoch 3, Train loss: 0.5711, Val loss: 0.9451, Val accy: 64.00%



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:48<00:00,  1.34s/it]

              precision    recall  f1-score   support

           A       0.65      0.67      0.66       344
           B       0.35      0.20      0.26       327
           C       0.42      0.58      0.49       326

    accuracy                           0.49       997
   macro avg       0.47      0.48      0.47       997
weighted avg       0.48      0.49      0.47       997

f1micro = 0.487 and f1macro = 0.468



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:46<00:00,  1.33s/it]

              precision    recall  f1-score   support

           A       0.67      0.66      0.67       344
           B       0.38      0.30      0.33       327
           C       0.43      0.53      0.47       326

    accuracy                           0.50       997
   macro avg       0.49      0.50      0.49       997
weighted avg       0.50      0.50      0.49       997

f1micro = 0.499 and f1macro = 0.492


Remove stop words

In [14]:
X_train = pre_process(X_train_sample,removeStopWords)
X_train.head()
X_test_stopWords = pre_process(X_test,removeStopWords)

In [15]:
model_stopWords = model.fit(X_train, y_train)
f1macro, f1micro = test_performance(model_stopWords, X_test, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")
f1macro, f1micro = test_performance(model_stopWords, X_test_stopWords, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.75s/it]

Epoch 1, Train loss: 1.1188, Val loss: 1.1037, Val accy: 33.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.72s/it]

Epoch 2, Train loss: 1.1075, Val loss: 1.1018, Val accy: 40.00%


...


Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:36<00:00,  1.25s/it]

              precision    recall  f1-score   support

           A       0.35      1.00      0.51       344
           B       0.00      0.00      0.00       327
           C       0.00      0.00      0.00       326

    accuracy                           0.35       997
   macro avg       0.12      0.33      0.17       997
weighted avg       0.12      0.35      0.18       997

f1micro = 0.345 and f1macro = 0.171



C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

              precision    recall  f1-score   support

           A       0.35      1.00      0.51       344
           B       0.00      0.00      0.00       327
           C       0.00      0.00      0.00       326

    accuracy                           0.35       997
   macro avg       0.12      0.33      0.17       997
weighted avg       0.12      0.35      0.18       997

f1micro = 0.345 and f1macro = 0.171



C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

In [16]:
X_train = pre_process(X_train_sample,removeStopWords)
X_train = pre_process(X_train,lemmatizer)

X_train.head()
X_test_All = pre_process(X_test,removeStopWords)
X_test_All = pre_process(X_test_stopWords,lemmatizer)

In [17]:
model_All = model.fit(X_train, y_train)
f1macro, f1micro = test_performance(model_All, X_test, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")
f1macro, f1micro = test_performance(model_All, X_test_All, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:38<00:00,  2.95s/it]

Epoch 1, Train loss: 1.1162, Val loss: 1.1009, Val accy: 32.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.71s/it]

Epoch 2, Train loss: 1.0519, Val loss: 1.0521, Val accy: 47.00%


...

Epoch 3, Train loss: 0.8259, Val loss: 1.0826, Val accy: 49.00%



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:50<00:00,  1.36s/it]

              precision    recall  f1-score   support

           A       0.44      0.85      0.58       344
           B       0.45      0.20      0.28       327
           C       0.42      0.25      0.31       326

    accuracy                           0.44       997
   macro avg       0.44      0.43      0.39       997
weighted avg       0.44      0.44      0.39       997

f1micro = 0.441 and f1macro = 0.391



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:57<00:00,  1.42s/it]

              precision    recall  f1-score   support

           A       0.49      0.67      0.56       344
           B       0.43      0.30      0.35       327
           C       0.41      0.37      0.39       326

    accuracy                           0.45       997
   macro avg       0.44      0.45      0.43       997
weighted avg       0.44      0.45      0.44       997

f1micro = 0.449 and f1macro = 0.434


In [20]:
X_train = pre_process(X_train_sample,stemmatizer)
X_test_stem = pre_process(X_test,stemmatizer)

model_stem = model.fit(X_train, y_train)
f1macro, f1micro = test_performance(model_stem, X_test, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")
f1macro, f1micro = test_performance(model_stem, X_test_stem, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")


Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:36<00:00,  2.83s/it]

Epoch 1, Train loss: 1.1144, Val loss: 1.0964, Val accy: 31.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:41<00:00,  3.22s/it]

Epoch 2, Train loss: 1.1053, Val loss: 1.0911, Val accy: 43.00%


...


Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:52<00:00,  1.38s/it]

              precision    recall  f1-score   support

           A       0.36      0.76      0.49       344
           B       0.00      0.00      0.00       327
           C       0.38      0.32      0.34       326

    accuracy                           0.37       997
   macro avg       0.25      0.36      0.28       997
weighted avg       0.25      0.37      0.28       997

f1micro = 0.366 and f1macro = 0.278



C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

              precision    recall  f1-score   support

           A       0.41      0.55      0.47       344
           B       0.00      0.00      0.00       327
           C       0.34      0.57      0.43       326

    accuracy                           0.38       997
   macro avg       0.25      0.37      0.30       997
weighted avg       0.25      0.38      0.30       997

f1micro = 0.375 and f1macro = 0.300



C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

In [25]:
from functools import partial

pipe_fn = partial(ngrams, n=2)

In [26]:
X_train = pre_process(X_train_sample,pipe_fn)
X_test_bigram = pre_process(X_test,pipe_fn)

model_bigram = model.fit(X_train, y_train)
f1macro, f1micro = test_performance(model_bigram, X_test, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")
f1macro, f1micro = test_performance(model_bigram, X_test_bigram, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:45<00:00,  3.51s/it]

Epoch 1, Train loss: 1.1226, Val loss: 1.0984, Val accy: 31.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:41<00:00,  3.17s/it]

Epoch 2, Train loss: 1.1084, Val loss: 1.0999, Val accy: 40.00%


...


Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [03:27<00:00,  1.66s/it]

              precision    recall  f1-score   support

           A       0.36      0.43      0.39       344
           B       0.34      0.61      0.44       327
           C       0.00      0.00      0.00       326

    accuracy                           0.35       997
   macro avg       0.23      0.35      0.28       997
weighted avg       0.24      0.35      0.28       997

f1micro = 0.349 and f1macro = 0.277



C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

              precision    recall  f1-score   support

           A       0.35      1.00      0.51       344
           B       0.00      0.00      0.00       327
           C       0.00      0.00      0.00       326

    accuracy                           0.35       997
   macro avg       0.12      0.33      0.17       997
weighted avg       0.12      0.35      0.18       997

f1micro = 0.345 and f1macro = 0.171



C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

In [28]:
pipe_fn = partial(ngrams, n=3)
X_train = pre_process(X_train_sample,pipe_fn)
X_test_trigram = pre_process(X_test,pipe_fn)

model_trigram = model.fit(X_train, y_train)
f1macro, f1micro = test_performance(model_trigram, X_test, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")
f1macro, f1micro = test_performance(model_trigram, X_test_trigram, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:38<00:00,  2.99s/it]

Epoch 1, Train loss: 1.1211, Val loss: 1.0921, Val accy: 40.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:39<00:00,  3.06s/it]

Epoch 2, Train loss: 1.1193, Val loss: 1.0970, Val accy: 40.00%


...


Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [03:03<00:00,  1.47s/it]

              precision    recall  f1-score   support

           A       0.35      0.99      0.51       344
           B       0.00      0.00      0.00       327
           C       0.35      0.02      0.03       326

    accuracy                           0.35       997
   macro avg       0.23      0.33      0.18       997
weighted avg       0.23      0.35      0.19       997

f1micro = 0.346 and f1macro = 0.182



C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

              precision    recall  f1-score   support

           A       0.35      1.00      0.51       344
           B       0.00      0.00      0.00       327
           C       0.00      0.00      0.00       326

    accuracy                           0.35       997
   macro avg       0.12      0.33      0.17       997
weighted avg       0.12      0.35      0.18       997

f1micro = 0.345 and f1macro = 0.171



C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

In [29]:
X_test.head()

,FalseSent,OptionA,OptionB,OptionC
0,"Summer in North America is great for skiing, ...",Snowmen can become violent due to their isolat...,You would not want to eat a violent snowman.,The temperature in North America during the su...
1,You can use detergent to dye your hair.,Detergent and bleach are both cleaning products.,Detergent and bleach cannot be used together.,Detergent isn't a hair product.
2,passing your driving license exams requires st...,the driving license is useless for studying fo...,driving license exams needs to study for driving,You cannot drive into classes even if you have...
3,The hangers bought the closet,A hanger is something you put your clothes on,Hangers are inanimate things which cannot buy ...,The hangers cannot eat
4,coffee depresses people,coffee inspires people,coffee with milk is rich,coffee with cake encourages


In [ ]:
model.bert_model = 'bert-base-cased'

model_cased = model.fit(X_train_sample, y_train)
f1macro, f1micro = test_performance(model_cased, X_test, y_test)
print(f"f1micro = {f1micro:.3f} and "f"f1macro = {f1macro:.3f}")

To save a model

In [ ]:
#save model to disk
savefile = 'BERT_TaskB.bin'
model.save(savefile)